In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import scipy.spatial
import matplotlib.pyplot as plt


# from transformers import T5Tokenizer, T5ForConditionalGeneration
from tqdm import tqdm
import pandas as pd
from torch.utils.data import DataLoader, SequentialSampler, TensorDataset
import torch
import pandas as pd
from pprint import pprint

import sys
import os
import glob


# Dataset

In [30]:
PATH = '/kaggle/input/medical-chatbot/train_data.csv'

In [31]:
dataset = pd.read_csv(PATH)
dataset.head()

,question,answer
0,what are the symptoms of hepatitis c?,"Fever, Fatigue, Joint pain, Muscle pain, Loss ..."
1,what are the symptoms of listeriosis?,"Fever, Muscle aches, Nausea, Diarrhea, Headach..."
2,what are the symptoms of ovarian cancer?,"Abdominal bloating, Pelvic pain, Difficulty ea..."
3,what are the symptoms of aortic stenosis?,"Shortness of breath, Chest pain or discomfort,..."
4,what are the symptoms of interstitial cystitis?,"Pelvic pain, Frequent urination, Urgency to ur..."


In [32]:
dataset.shape

(2216, 2)

In [14]:
!pip install simplet5

In [35]:
dataset.head()

,question,answer
0,what are the symptoms of hepatitis c?,"Fever, Fatigue, Joint pain, Muscle pain, Loss ..."
1,what are the symptoms of listeriosis?,"Fever, Muscle aches, Nausea, Diarrhea, Headach..."
2,what are the symptoms of ovarian cancer?,"Abdominal bloating, Pelvic pain, Difficulty ea..."
3,what are the symptoms of aortic stenosis?,"Shortness of breath, Chest pain or discomfort,..."
4,what are the symptoms of interstitial cystitis?,"Pelvic pain, Frequent urination, Urgency to ur..."


In [36]:
 df = dataset

In [37]:
q_list =df['question']                          # questions list to feed the model
n_list = df['answer'] +" </s>"  # answers list to feed the model

dict_data = {'source_text': q_list,
      'target_text': n_list}

df = pd.DataFrame(dict_data)
df.head()      

,source_text,target_text
0,what are the symptoms of hepatitis c?,"Fever, Fatigue, Joint pain, Muscle pain, Loss ..."
1,what are the symptoms of listeriosis?,"Fever, Muscle aches, Nausea, Diarrhea, Headach..."
2,what are the symptoms of ovarian cancer?,"Abdominal bloating, Pelvic pain, Difficulty ea..."
3,what are the symptoms of aortic stenosis?,"Shortness of breath, Chest pain or discomfort,..."
4,what are the symptoms of interstitial cystitis?,"Pelvic pain, Frequent urination, Urgency to ur..."


In [38]:
df['source_text'][0]

'what are the symptoms of hepatitis c?'

In [39]:
df['target_text'][0]

'Fever, Fatigue, Joint pain, Muscle pain, Loss of appetite, Nausea, Vomiting, Abdominal pain, Dark urine, Jaundice </s>'

In [40]:
df.shape, len(df.source_text.unique()), len(df.target_text.unique())

((2216, 2), 760, 749)

In [41]:
# splitting data into train and test data
train_data, val_data = train_test_split(df[:-100], test_size=0.2)
test_data = df[-100:]
train_data.shape, val_data.shape, test_data.shape

((1692, 2), (424, 2), (100, 2))

In [75]:
train_data['source_text'] = train_data['source_text'].astype(str)
train_data['target_text'] = train_data['target_text'].astype(str)

val_data['source_text'] = val_data['source_text'].astype(str)
val_data['target_text'] = val_data['target_text'].astype(str)

# Then try training
from simplet5 import SimpleT5
model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")
model.train(train_df=train_data,
           eval_df=val_data, 
           source_max_token_len=128, 
           target_max_token_len=50, 
           batch_size=8,
           max_epochs=10,
           use_gpu=True)

Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [76]:
ls ./outputs

simplet5-epoch-0-train-loss-2.167-val-loss-1.5443/
simplet5-epoch-1-train-loss-1.6523-val-loss-1.2815/
simplet5-epoch-2-train-loss-1.3681-val-loss-1.0959/
simplet5-epoch-3-train-loss-1.1528-val-loss-0.9246/
simplet5-epoch-4-train-loss-0.9754-val-loss-0.7903/
simplet5-epoch-5-train-loss-0.8232-val-loss-0.6649/
simplet5-epoch-6-train-loss-0.695-val-loss-0.5842/
simplet5-epoch-7-train-loss-0.5837-val-loss-0.5029/
simplet5-epoch-8-train-loss-0.491-val-loss-0.4532/
simplet5-epoch-9-train-loss-0.4146-val-loss-0.4159/


In [79]:
# let's load the trained model for inferencing
model.load_model("t5","./outputs/simplet5-epoch-9-train-loss-0.4146-val-loss-0.4159/", use_gpu=True)

In [80]:
test_data['source_text']

2116         what are the symptoms of multiple sclerosis?
2117               what are the symptoms of strep throat?
2118            what are the treatments for osteoporosis?
2119               what are the symptoms of ehrlichiosis?
2120      what are the symptoms of myocardial infarction?
                              ...                        
2211       what are the treatments for cryptosporidiosis?
2212            what are the treatments for brain tumors?
2213       what are the treatments for colorectal cancer?
2214    what are the symptoms of nonalcoholic fatty li...
2215                  what are the symptoms of influenza?
Name: source_text, Length: 100, dtype: object

In [81]:
q_test = test_data['source_text'][2116]
q_ans = test_data['target_text'][2116]

print("Question: ", q_test)
print('-'*50)
print("Answer: ",q_ans)

Question:  what are the symptoms of multiple sclerosis?
--------------------------------------------------
Answer:  Fatigue, Numbness or tingling in the limbs, Muscle weakness or spasms, Blurred or double vision, Difficulty with coordination and balance, Problems with speech and swallowing </s>


In [82]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.device = device

# Move model to the specified device
model.model.to(device)

# Make sure q_test is on the same device if it's a tensor
# If q_test is a string or list of strings, we don't need to move it
if isinstance(q_test, torch.Tensor):
    q_test = q_test.to(device)

# Now try prediction
predicted_ans = model.predict(q_test)[0]
print(predicted_ans)

Fatigue, Weakness, Loss of appetite, Difficulty speaking and swallowing, Muscle weakness or spasms, Difficulty with coordination and balance, Difficulty with speech and swallowing


# Evaluation data

In [83]:
y_test = [a for a in test_data['target_text'] ]

# get the predicted answers
y_pred = []
for q in test_data['source_text']:
    y_pred.append(model.predict(q)[0])

**Sample of evaluation**

In [84]:
pprint(f'Question is:\n {q_test}')
pprint(f'Answer is:\n {q_ans}')
pprint(f'Predicted answer is:\n {predicted_ans}')

'Question is:\n what are the symptoms of multiple sclerosis?'
('Answer is:\n'
 ' Fatigue, Numbness or tingling in the limbs, Muscle weakness or spasms, '
 'Blurred or double vision, Difficulty with coordination and balance, Problems '
 'with speech and swallowing </s>')
('Predicted answer is:\n'
 ' Fatigue, Weakness, Loss of appetite, Difficulty speaking and swallowing, '
 'Muscle weakness or spasms, Difficulty with coordination and balance, '
 'Difficulty with speech and swallowing')


## [ROUGE score ](https://pypi.org/project/rouge-score/)

In [65]:
!pip install rouge-score

  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24956 sha256=dbd6075788def5f848a55d8c0a9560196372736eb15ee3f6674f87abaebf0b3b
  Stored in directory: /root/.cache/pip/wheels/84/ac/6b/38096e3c5bf1dc87911e3585875e21a3ac610348e740409c76
Successfully built rouge-score


In [85]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
scores = scorer.score('The quick brown fox jumps over the lazy dog',
                      'The quick brown dog jumps on the log.')
pprint(scores)

{'rouge1': Score(precision=0.75, recall=0.6666666666666666, fmeasure=0.7058823529411765),
 'rougeL': Score(precision=0.625, recall=0.5555555555555556, fmeasure=0.5882352941176471)}


**Sample of evaluation**

In [70]:
pprint(f'Question is: {q_test}')
print('-'*50)
pprint(f'Ansewer is: {q_ans}')
print('-'*50)
pprint(f'Predicted answer is: {predicted_ans}')

'Question is: what are the treatments for colorectal cancer?'
--------------------------------------------------
('Ansewer is: Surgery to remove the cancerous tissue, Radiation therapy, '
 'Chemotherapy, Immunotherapy </s>')
--------------------------------------------------
('Predicted answer is: Surgery to remove the cancer, Radiation therapy, '
 'Chemotherapy')


In [87]:
from rouge_score import rouge_scorer

# Initialize the scorer with all metrics
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL', 'rougeLsum'], use_stemmer=True)

# Initialize lists for all metrics
scores = {
    'rouge1': {'precision': [], 'recall': [], 'fmeasure': []},
    'rouge2': {'precision': [], 'recall': [], 'fmeasure': []},
    'rougeL': {'precision': [], 'recall': [], 'fmeasure': []},
    'rougeLsum': {'precision': [], 'recall': [], 'fmeasure': []}
}

# Calculate scores for each prediction
for i in range(len(y_pred)):
    # Get scores for current prediction
    current_scores = scorer.score(y_test[i], y_pred[i])
    
    # Store all metrics
    for metric in ['rouge1', 'rouge2', 'rougeL', 'rougeLsum']:
        scores[metric]['precision'].append(round(current_scores[metric][0], 3))
        scores[metric]['recall'].append(round(current_scores[metric][1], 3))
        scores[metric]['fmeasure'].append(round(current_scores[metric][2], 3))

# Calculate and print average scores
print("Average ROUGE scores:")
for metric in scores:
    print(f"\n{metric.upper()}:")
    for score_type in scores[metric]:
        avg_score = sum(scores[metric][score_type]) / len(scores[metric][score_type])
        print(f"{score_type}: {round(avg_score, 3)}")

Average ROUGE scores:

ROUGE1:
precision: 0.724
recall: 0.571
fmeasure: 0.626

ROUGE2:
precision: 0.594
recall: 0.471
fmeasure: 0.516

ROUGEL:
precision: 0.705
recall: 0.557
fmeasure: 0.61

ROUGELSUM:
precision: 0.705
recall: 0.557
fmeasure: 0.61


In [90]:
!zip -r file.zip /kaggle/working/outputs/simplet5-epoch-9-train-loss-0.4146-val-loss-0.4159

  adding: kaggle/working/outputs/simplet5-epoch-9-train-loss-0.4146-val-loss-0.4159/ (stored 0%)
  adding: kaggle/working/outputs/simplet5-epoch-9-train-loss-0.4146-val-loss-0.4159/config.json (deflated 63%)
  adding: kaggle/working/outputs/simplet5-epoch-9-train-loss-0.4146-val-loss-0.4159/tokenizer.json (deflated 59%)
  adding: kaggle/working/outputs/simplet5-epoch-9-train-loss-0.4146-val-loss-0.4159/spiece.model (deflated 48%)
  adding: kaggle/working/outputs/simplet5-epoch-9-train-loss-0.4146-val-loss-0.4159/special_tokens_map.json (deflated 83%)
  adding: kaggle/working/outputs/simplet5-epoch-9-train-loss-0.4146-val-loss-0.4159/pytorch_model.bin (deflated 8%)
  adding: kaggle/working/outputs/simplet5-epoch-9-train-loss-0.4146-val-loss-0.4159/tokenizer_config.json (deflated 80%)


In [91]:
from IPython.display import FileLink 
FileLink(r'file.zip')

/kaggle/working/file.zip

In [ ]:
/kaggle/working/lightning_logs/version_3/events.out.tfevents.1738585646.ad9e2fc2f114.22.3